Code here is adapted from [this Colab notebook](https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb#scrollTo=ktegpM42AooT).

In [9]:
# Create a dataset YAML file for fine-tuning YOLOv8 on custom data

DATASET_YAML_FILENAME = "fine_tune_yolov8_config.yaml"

dataset_yaml_content = """\
path: data/ultralytics_yolo_format
train: images/train
val: images/val

names:
  0: chess board screenshot
"""

with open(DATASET_YAML_FILENAME, "w") as f:
    f.write(dataset_yaml_content)

In [2]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.65  Python-3.11.9 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)
Setup complete  (8 CPUs, 39.8 GB RAM, 843.1/929.7 GB disk)


In [4]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  import comet_ml; comet_ml.login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

In [27]:
!yolo train model=yolov8n.pt data=fine_tune_yolov8_config.yaml epochs=6 imgsz=640

Ultralytics YOLOv8.2.65 🚀 Python-3.11.9 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=fine_tune_yolov8_config.yaml, epochs=6, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: Native output logging mode is not available, falling back to basic output logging
COMET INFO: Experiment is live on comet.com https://www.comet.com/jacksonthall22/general/f79ed75053d648f6b542f4a880698141


train: Scanning C:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\fine_tuning\data\ultralytics_yolo_format\labels\train.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]
train: Scanning C:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\fine_tuning\data\ultralytics_yolo_format\labels\train.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

val: Scanning C:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\fine_tuning\data\ultralytics_yolo_format\labels\val.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 

In [34]:
from ultralytics import YOLO

# model = YOLO('..\runs\detect\train8\weights\best.pt')
model = YOLO('yolov8n_best.pt')
model.val(data='fine_tune_yolov8_config.yaml')

Ultralytics YOLOv8.2.65  Python-3.11.9 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\fine_tuning\data\ultralytics_yolo_format\labels\val.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:14<00:00,  2.16it/s]


                   all        500        500          1          1      0.995      0.995
Speed: 1.0ms preprocess, 8.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to c:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\runs\detect\val4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024B9164D0D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [40]:
import os
import random

val_images_dir = r"data\ultralytics_yolo_format\images\val"
val_images = os.listdir(val_images_dir)

image_idx = random.randint(0, len(val_images))
result = model(os.path.join(val_images_dir, val_images[image_idx]))
result[0].show()


image 1/1 c:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\fine_tuning\data\ultralytics_yolo_format\images\val\002021.png: 448x640 1 chess board screenshot, 52.4ms
Speed: 292.4ms preprocess, 52.4ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
